# μPC

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thebuckleylab/jpc/blob/main/examples/mupc.ipynb)

This notebook demonstrates how to train residual networks with [**μPC**](https://arxiv.org/abs/2505.13124), a reparameterisation of PC that allows stable training of very deep (100+ layer) networks while also enabling zero-shot hyperparameter transfer (see [Innocenti et al., 2025](https://arxiv.org/abs/2505.13124)).

In [1]:
%%capture
!pip install torch==2.3.1
!pip install torchvision==0.18.1

In [2]:
import jpc

import jax.random as jr
import equinox as eqx
import equinox.nn as nn
import optax

from jax.tree_util import tree_leaves
from typing import List

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import warnings
warnings.simplefilter('ignore')  # ignore warnings

## Hyperparameters

We define some global parameters, including the network architecture, learning rate, batch size, etc. We choose a network with "only" 30 layers and 128 hidden neurons so that it can run relatively fast on a CPU, but feel free to try deeper and wider networks.

In [3]:
SEED = 0

INPUT_DIM = 784
WIDTH = 128
DEPTH = 30
OUTPUT_DIM = 10
ACT_FN = "relu"

ACTIVITY_LR = 5e-1
PARAM_LR = 1e-1
BATCH_SIZE = 64
TEST_EVERY = 100
N_TRAIN_ITERS = 900

## Dataset

Some utils to fetch MNIST.

In [4]:
def get_mnist_loaders(batch_size):
    train_data = MNIST(train=True, normalise=True)
    test_data = MNIST(train=False, normalise=True)
    train_loader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True
    )
    test_loader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True
    )
    return train_loader, test_loader


class MNIST(datasets.MNIST):
    def __init__(self, train, normalise=True, save_dir="data"):
        if normalise:
            transform = transforms.Compose(
                [
                    transforms.ToTensor(),
                    transforms.Normalize(
                        mean=(0.1307), std=(0.3081)
                    )
                ]
            )
        else:
            transform = transforms.Compose([transforms.ToTensor()])
        super().__init__(save_dir, download=True, train=train, transform=transform)

    def __getitem__(self, index):
        img, label = super().__getitem__(index)
        img = torch.flatten(img)
        label = one_hot(label)
        return img, label


def one_hot(labels, n_classes=10):
    arr = torch.eye(n_classes)
    return arr[labels]
    

## Creating a μPC model

To parameterise a model with μPC, one can use a few convenience functions of `jpc` to create an MLP or fully connected network with [`jpc.make_mlp()`](https://thebuckleylab.github.io/jpc/api/Utils/#jpc.make_mlp) and an associated skip model with [`jpc.make_skip model()`](https://thebuckleylab.github.io/jpc/api/Utils/#jpc.make_skip_model). Note that μPC works only for a specific type of ResNet, namely one with one-layer skip connections at every layer except from the input to the next layer and from the penultimate layer to the output (see [Innocenti et al., 2025](https://arxiv.org/abs/2505.13124)), as shown below.

In [5]:
key = jr.PRNGKey(SEED)

# MLP
model = jpc.make_mlp(
    key,
    input_dim=INPUT_DIM,
    width=WIDTH,
    depth=DEPTH,
    output_dim=OUTPUT_DIM,
    act_fn=ACT_FN,
    param_type="mupc"
)

# skip model
skip_model = jpc.make_skip_model(DEPTH)

At training and test time we would need to pass both models to relevant `jpc` functions and change the argument `param_type = "mupc"` (default is `"sp"` for standard parameterisation). 

Alternatively, one could define a model class embedding the parameterisation itself and leave the above arguments to their default. This solution is more elegant but it can be harder to debug, at least for a fully connected architecture. However, if you would like to experiment with different parameterisations and more complex architectures (e.g. CNNs), we recommend this approach. 

## Train and test

For training, we use the [advanced API](https://thebuckleylab.github.io/jpc/advanced_usage/) including the functions [`jpc.init_activities_with_ffwd()`](https://thebuckleylab.github.io/jpc/api/Initialisation/#jpc.init_activities_with_ffwd) to initialise the activities, [`jpc.update_activities()`](https://thebuckleylab.github.io/jpc/api/Discrete%20updates/#jpc.update_activities) to perform PC inference, and [`jpc.update_params()`](https://thebuckleylab.github.io/jpc/api/Discrete%20updates/#jpc.update_params) to update the weights. All these functions accept `skip_model` and `param_type` as arguments. Note, however, that one can replace these functions with [`jpc.make_pc_step()`](https://thebuckleylab.github.io/jpc/api/Training/#jpc.make_pc_step). For testing, we use [`jpc.test_discriminative_pc()`](https://thebuckleylab.github.io/jpc/api/Testing/#jpc.test_discriminative_pc).

In [6]:
def evaluate(model, skip_model, test_loader, param_type):
    avg_test_acc = 0
    for _, (img_batch, label_batch) in enumerate(test_loader):
        img_batch, label_batch = img_batch.numpy(), label_batch.numpy()

        _, test_acc = jpc.test_discriminative_pc(
            model=model,
            input=img_batch,
            output=label_batch,
            skip_model=skip_model,
            param_type=param_type
        )
        avg_test_acc += test_acc

    return avg_test_acc / len(test_loader)


def train(
      model,
      skip_model,
      param_type,
      activity_lr,  
      param_lr,
      batch_size,
      test_every,
      n_train_iters
):  
    activity_optim = optax.sgd(activity_lr)
    param_optim = optax.adam(param_lr)
    param_opt_state = param_optim.init(
        (eqx.filter(model, eqx.is_array), skip_model)
    )
    train_loader, test_loader = get_mnist_loaders(batch_size)

    for iter, (img_batch, label_batch) in enumerate(train_loader):
        img_batch, label_batch = img_batch.numpy(), label_batch.numpy()

        # initialise activities
        activities = jpc.init_activities_with_ffwd(
            model=model,
            input=img_batch,
            skip_model=skip_model,
            param_type=param_type
        )
        activity_opt_state = activity_optim.init(activities)
        train_loss = jpc.mse_loss(activities[-1], label_batch)

        # inference
        for t in range(len(model)):
            activity_update_result = jpc.update_activities(
                params=(model, skip_model),
                activities=activities,
                optim=activity_optim,
                opt_state=activity_opt_state,
                output=label_batch,
                input=img_batch,
                param_type=param_type
            )
            activities = activity_update_result["activities"]
            activity_opt_state = activity_update_result["opt_state"]

        # learning
        param_update_result = jpc.update_params(
            params=(model, skip_model),
            activities=activities,
            optim=param_optim,
            opt_state=param_opt_state,
            output=label_batch,
            input=img_batch,
            param_type=param_type
        )
        model = param_update_result["model"]
        skip_model = param_update_result["skip_model"]
        param_opt_state = param_update_result["opt_state"]

        if np.isinf(train_loss) or np.isnan(train_loss):
            print(
                f"Stopping training because of divergence, train loss={train_loss}"
            )
            break
    
        if ((iter+1) % test_every) == 0:
            avg_test_acc = evaluate(
                model=model,
                skip_model=skip_model, 
                test_loader=test_loader, 
                param_type=param_type
            )
            print(
                f"Train iter {iter+1}, train loss={train_loss:4f}, "
                f"avg test accuracy={avg_test_acc:4f}"
            )
            if (iter+1) >= n_train_iters:
                break


## Run

Note that on a CPU the script below should take about a minute to complete.

In [7]:
train(
    model=model,
    skip_model=skip_model,
    param_type="mupc",
    activity_lr=ACTIVITY_LR,
    param_lr=PARAM_LR,
    batch_size=BATCH_SIZE,
    test_every=TEST_EVERY,
    n_train_iters=N_TRAIN_ITERS
)

Train iter 100, train loss=0.014857, avg test accuracy=86.718750
Train iter 200, train loss=0.012227, avg test accuracy=89.352966
Train iter 300, train loss=0.012027, avg test accuracy=91.065704
Train iter 400, train loss=0.010815, avg test accuracy=91.766830
Train iter 500, train loss=0.010547, avg test accuracy=92.497993
Train iter 600, train loss=0.008235, avg test accuracy=92.738380
Train iter 700, train loss=0.009772, avg test accuracy=93.199120
Train iter 800, train loss=0.010498, avg test accuracy=93.699921
Train iter 900, train loss=0.009273, avg test accuracy=93.509613


For comparison, try to change to the standard parameterisation with `param_type = "sp"`. If you are using your own μPC-parameterised model class, then you can leave the default `skip_model = None` and `param_type = "sp"`.